In [1]:
#Import Libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import seaborn as sns

pd.options.mode.chained_assignment = None  
pd.set_option('display.max_columns',80)  # or 1000
pd.set_option('display.max_rows', 100)  # or 1000

from sklearn import preprocessing


import ast #For converting string to list

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Import dataset
df_raw = pd.read_csv('listings.csv')

#Checking Shape
print(f'{df_raw.shape[0]} examples, {df_raw.shape[1]} features')

# ... and duplicates
print("It contains {} duplicates.".format(df_raw.duplicated().sum()))


24551 examples, 106 features
It contains 0 duplicates.


In [3]:
# Deciding which columns to work with

#1 City, street, state are mostly same for all, id is unique and scrape_id is same for all.
#2 experiences_offered, thumbnail_url, medium_url are  empty
#3 picture_url, host_thumbnail_url, host_picture_url links are broken.

columns_to_discard = ['city','scrape_id','experiences_offered','thumbnail_url','medium_url',
                     'picture_url','xl_picture_url','host_acceptance_rate','host_thumbnail_url',
                     'host_picture_url','street','state','market','smart_location','country_code',
                     'country','has_availability','calendar_last_scraped','license','jurisdiction_names',
                     'is_business_travel_ready','host_name','last_scraped','square_feet',
                     'calendar_updated','host_id']

#Columns with working link: Can be used for data scraping
linked_columns= ['listing_url','host_url']

#Columns that can be analyzed with NLP
text_columns = ['name','summary','space','description','neighborhood_overview',
               'notes','transit','access','interaction','house_rules','host_about']

#Date columns can be feature-engineered
date_columns = ['host_since','first_review','last_review',]

#Categorical columns (will be converted to categories)
cat_columns = ['last_scraped','host_response_time','host_response_rate','host_is_superhost',
              'host_neighbourhood','host_verifications','host_has_profile_pic','host_identity_verified',
              'neighbourhood','neighbourhood_cleansed','neighbourhood_group_cleansed','is_location_exact',
              'property_type','room_type','bed_type','amenities','calendar_updated','requires_license',
              'instant_bookable','cancellation_policy','require_guest_profile_picture',
               'require_guest_phone_verification']

target_column = 'price'

print(len(df_raw.columns),'total column count\n')
print(len(columns_to_discard), 'useless columns')
print(len(linked_columns),'columns with link included')
print(len(text_columns),'text columns')
print(len(date_columns),'date type columns')
print(len(cat_columns),' categorical columns')

106 total column count

26 useless columns
2 columns with link included
11 text columns
3 date type columns
22  categorical columns


### Notes:

* Interesting feature is text german or english
* Didn't understand host_listings_count and host_total_listings_count (will look more closely)
They are almost same (only 51 of them different from 24551)
* Zipcode should be investigated more (to understand the correlation between numbers and location
if there is any.)
* How bathrooms can be a decimal number like 2.5 =)
* Square_feet, weekly/monthly prices are mostly NaN
* What is the difference between minimum_nights vs minimum_minimum_nights (23748 of them are same)
* Interesting visualizations for review columns (Average is not 5 for example. So we may shift the average)
* host_neighbourhood can be processed as inside the city vs outside the city. (Need Berlin's neighbourhoods)


In [4]:
#Check if there is any nan value in price
print(df_raw.price.isna().sum())
print('Price data doesnt have any null, which means we can use the full dataset for prediction model')

0
Price data doesnt have any null, which means we can use the full dataset for prediction model


In [5]:
#Dropping useless columns
def drop_useless_columns(df):
    temp = df.copy()   
    temp= temp.drop(columns= columns_to_discard)
    
    print('shape of df:',temp.shape)
    print(len(columns_to_discard),'useless columns are dropped')
    
    return temp

df_raw = drop_useless_columns(df_raw)

shape of df: (24551, 80)
26 useless columns are dropped


In [6]:
#Dropping text columns ( will handle them seperately after)
def drop_text_columns(df):
    temp = df.copy()
    temp_columns = text_columns + linked_columns
    temp = temp.drop(columns = temp_columns)

    print('shape of df:',temp.shape)
    print(len(temp_columns),'text columns are dropped')
    
    return temp

df_raw = drop_text_columns(df_raw)

shape of df: (24551, 67)
13 text columns are dropped


## Preprocessing the Data


In [7]:
#Finding NaN values
def print_nan_counts(df):
    print('Column Name: NaN Count')
    print('-----------------------')
    for column in df.columns:
        if any(df[column].isna()):
            print(column,':' ,df[column].isna().sum())

print_nan_counts(df_raw)

Column Name: NaN Count
-----------------------
host_since : 51
host_location : 143
host_response_time : 10452
host_response_rate : 10452
host_is_superhost : 51
host_neighbourhood : 5186
host_listings_count : 51
host_total_listings_count : 51
host_has_profile_pic : 51
host_identity_verified : 51
zipcode : 556
bathrooms : 29
bedrooms : 19
beds : 31
weekly_price : 21248
monthly_price : 22175
security_deposit : 9624
cleaning_fee : 7280
first_review : 4328
last_review : 4328
review_scores_rating : 4819
review_scores_accuracy : 4845
review_scores_cleanliness : 4842
review_scores_checkin : 4864
review_scores_communication : 4847
review_scores_location : 4863
review_scores_value : 4866
cancellation_policy : 1
reviews_per_month : 4328


I decided to get rid of that 51 rows since in most of the columns they don't have any information. 6 columns are dropped.

In [8]:
#Get rid of that 51 empty rows:
df_raw = df_raw[df_raw['host_since'].isna() == False]

print('shape of df:',df_raw.shape)

shape of df: (24500, 67)


In [9]:
#intersection of two lists Return true if there is at least one intersection
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return len(lst3) != 0

In [10]:
#Categorize Host Location as Berlin, Outside of Berlin, Outside of Germany and Unknown
def categorize_host_location(df):
    
    temp = df.copy() 
    locations = []
    counter = 0
    words =['Germany','DE']

    for _,row in temp.iterrows():

        location = row['host_location']    
        location= str(location)
        word_list = location.split(', ')

        if 'Berlin' in word_list:
            locations.append('Berlin')

        elif intersection(words,word_list):
            locations.append('Outside-of-Berlin')
            
        elif word_list[0] =='nan':
            locations.append('Unknown')
        else:
            #print(location)
            locations.append('Outside of Germany')

    temp['host_location'] = locations       

    return temp


df_raw = categorize_host_location(df_raw)

In [11]:
#Categorize Host Neighbourhood get most popular 15 and labels others as "others"
def categorize_host_neighbourhood(df):
    temp = df.copy()
    popular_neighbourhoods = ['Neukölln', 'Prenzlauer Berg','Kreuzberg','Friedrichshain',
      'Mitte','Wedding','Schöneberg','Charlottenburg','Moabit',
      'Wilmersdorf','Pankow','Tempelhof','Rummelsburg','Weißensee','Tiergarten']
    
    hood_list=[]
    
    for hood in temp['host_neighbourhood']:
        if hood in popular_neighbourhoods:
            hood_list.append(hood)
        else:
            hood_list.append('other')
    
    temp['host_neighbourhood'] = hood_list
    
    return temp

df_raw = categorize_host_neighbourhood(df_raw)

In [12]:
#Replace NaN values with 0:
def replace_nan(df,column):
    temp = df.copy()
    temp[column] = temp[column].fillna('$0.00')
    
    return temp

#Probably they are not charging cleaning fee and security deposits that's why they are nan.
df_raw = replace_nan(df_raw,'cleaning_fee')
df_raw = replace_nan(df_raw,'security_deposit')

In [13]:
print_nan_counts(df_raw)

Column Name: NaN Count
-----------------------
host_response_time : 10401
host_response_rate : 10401
zipcode : 555
bathrooms : 29
bedrooms : 19
beds : 31
weekly_price : 21209
monthly_price : 22134
first_review : 4313
last_review : 4313
review_scores_rating : 4802
review_scores_accuracy : 4828
review_scores_cleanliness : 4825
review_scores_checkin : 4847
review_scores_communication : 4830
review_scores_location : 4846
review_scores_value : 4849
cancellation_policy : 1
reviews_per_month : 4313


I'm planning to use XgBoost for the model and since XgBoost has the capability of handling missing data during training.

In [14]:
def convert_prices_into_float(df):
    temp = df.copy()
    
    price_columns = ['price','weekly_price','monthly_price','cleaning_fee','security_deposit','extra_people']
    for column in price_columns:
        temp[column] = temp[column].str.replace('$','').str.replace(',', '').astype(float)
        
    return temp

df_raw= convert_prices_into_float(df_raw)

In [15]:
 def categorize_columns(columns,df):
        le = preprocessing.LabelEncoder()
        temp = df.copy()
        
        for feature in columns:
            temp[feature] = le.fit_transform(temp[feature])
            
        return temp

In [16]:
#Columns to categorize
col_to_cat= ['host_location','host_response_time','host_is_superhost',
             'host_neighbourhood','cancellation_policy',
             'neighbourhood_cleansed','neighbourhood_group_cleansed',
             'require_guest_phone_verification','require_guest_profile_picture',
             'instant_bookable','requires_license','room_type','bed_type',
             'is_location_exact']

df_raw= categorize_columns(col_to_cat,df_raw)

In [17]:
def convert_string_to_list(string):
    return ast.literal_eval(string)

def create_verification_features(df):
    temp = df.copy()
    verifications = ['email','phone','facebook','reviews','government_id','selfie']
    
    temp.host_verifications = temp.host_verifications.apply(convert_string_to_list)

    for verification in verifications:
        temp[verification] = [True if verification in x else False for x in temp.host_verifications]
        
    temp.drop(columns=['host_verifications'],inplace=True)
        
    return temp

In [18]:
df_raw = create_verification_features(df_raw)

In [22]:
df_raw.host_has_profile_pic.value_counts()

t    24430
f       70
Name: host_has_profile_pic, dtype: int64

In [19]:
print('Columns with "object" type :')
[ column for column in df_raw.columns if df_raw[column].dtype == object ]

Columns with "object" type :


['host_since',
 'host_response_rate',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'zipcode',
 'is_location_exact',
 'property_type',
 'amenities',
 'first_review',
 'last_review']

In [20]:
df_raw1.reset_index(drop=True,inplace= True)

NameError: name 'df_raw1' is not defined